In [19]:
select = 29

# up through 29

import SimpleITK as sitk

#%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, fixed
import os
import gui
import resample_itk_image
from myshow import myshow, myshow3d
from itertools import chain

import pandas as pd

%matplotlib widget

# PC
data_folder = r"C:\Users\david\UW\Ryan Kellogg - Kempe files\subjectData"
data_folder = os.path.abspath(os.path.expanduser(os.path.expandvars(data_folder)))

# create output directories if they don't exist

#for root, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
#    if root[len(data_folder):].count(os.sep) == 2:
#        if not (os.path.exists(os.path.join(root,'output'))):
#            os.mkdir(os.path.join(root,'output'))

# generate structure of files 

data_subj_list = list()
output_dir_list = list()
for data_subj, dirs, files in os.walk(data_folder, topdown=True):
    # walking tips from https://stackoverflow.com/questions/42720627/python-os-walk-to-certain-level
    if (data_subj[len(data_folder):].count(os.sep) == 3) and not 'output' in data_subj[len(data_folder):]:
       # print(data_subj)
        super_folder = data_subj.rsplit('\\',1)
        OUTPUT_DIR = os.path.join(super_folder[0],'output')
        
        data_subj_list.append(data_subj)
        output_dir_list.append(OUTPUT_DIR)

length_data = len(data_subj_list)



# Read the original series. First obtain the series file names using the
# image series reader.
data_subj = data_subj_list[select]
OUTPUT_DIR = output_dir_list[select]
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_subj)
if not series_IDs:
    print("ERROR: given directory \""+data_folder+"\" does not contain a DICOM series.")

series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_subj, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time).
# By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the
# private ones.
series_reader.MetaDataDictionaryArrayUpdateOn()
#series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()

reader = sitk.ImageFileReader()
reader.SetFileName(series_file_names[0])
#reader.LoadPrivateTagsOn();

reader.ReadImageInformation();

#for k in reader.GetMetaDataKeys():
#    v = reader.GetMetaData(k)
#    print("({0}) = = \"{1}\"".format(k,v))

print('Original Image Spacing ',image3D.GetSpacing())
print('Original Image Size ', image3D.GetSize())

orig_size = image3D.GetSize()
orig_spacing = image3D.GetSpacing()
min_orig = min(orig_spacing)
max_orig = max(orig_spacing)

intermed_spacing = 2.0
image3D_resample = resample_itk_image.resample_img(image3D,[intermed_spacing,intermed_spacing,intermed_spacing])

print('Resampled Image Spacing ',image3D_resample.GetSpacing())
print('Resampled Image Size ', image3D_resample.GetSize())

# recast types as necessary for analysis 

recast_image3D = sitk.Cast(sitk.RescaleIntensity(image3D_resample), sitk.sitkUInt8)
recast32_image3D = sitk.Cast(image3D_resample, sitk.sitkFloat32)

#
#img = sitk.ReadImage(series_file_names)

# plot halfway 
#z = int(image3D.GetDepth()/2)
#plt.imshow(sitk.GetArrayViewFromImage(image3D)[z,:,:], cmap=plt.cm.Greys_r)
#plt.axis('off');

#intensity_values = sitk.GetArrayViewFromImage(image3D)
#values_list = list(intensity_values)
#values_flattened = list(chain.from_iterable(values_list))
#values_flattened = np.array(values_flattened).flatten()

#if plot_values_hist:
#    plt.figure()
#    plt.hist(values_flattened, bins=100)
#    plt.title("Intensity Values in ROI")
#    plt.show()      

# select points
point_acquisition_interface = gui.PointDataAquisition(image3D_resample)

Original Image Spacing  (0.488281, 0.488281, 5.256999969482422)
Original Image Size  (512, 512, 32)
Resampled Image Spacing  (2.0, 2.0, 2.0)
Resampled Image Size  (125, 125, 84)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
initial_seed_point_indexes = point_acquisition_interface.get_point_indexes()


# implicit thresholds 

#seg_implicit_thresholds = sitk.ConfidenceConnected(image3D, seedList=initial_seed_point_indexes,
#                                                   numberOfIterations=1,
#                                                   multiplier=2,
#                                                   initialNeighborhoodRadius=5,
#                                                   replaceValue=1)

#seg_implicit_thresholds = sitk.ConnectedThreshold(image3D_resample, seedList=initial_seed_point_indexes, lower=400, upper=1700)

#gui.MultiImageDisplay(image_list = [sitk.LabelOverlay(recast_image3D, seg_implicit_thresholds)],title_list = ['confidence connected result'])

#myshow3d(seg_implicit_thresholds)

#seg = sitk.Image(image3D_resample.GetSize(), sitk.sitkUInt8)
#seg.CopyInformation(image3D_resample)
#for index in range(1,len(initial_seed_point_indexes)):
#    seg[initial_seed_point_indexes[index]] = 1
#seg = sitk.BinaryDilate(seg, 7)

#myshow(sitk.LabelOverlay(recast_image3D, seg))


# write points used to file for validation 
initial_seed_point_indexes
np.savetxt(os.path.join(OUTPUT_DIR,'seeds.csv'),initial_seed_point_indexes,delimiter=',')
